In [3]:
import numpy as np

"""STVM 계산(3차로 점선)"""
import pandas as pd

import os


# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 5400

Q_critical = 2200

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1,
    "w7" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
###################################################################################################################

# 함수 모음
###################################################################################################################
# 평균속도
def speed_mean(df):
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    return speed_mean_df

# 밀도
def density_mean(df):
    density_mean_df = df.assign(K = df["V_count"] * 12 / df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    return density_mean_df

# 가중 중차량 비율
def heavy_rate(original_df):

    # 진입부 검지기 선정
    measurement = 10

    # TimeGroup별 중차량 갯수 집계
    heavy_df = (
        original_df[
            (original_df["New_Measurement"] == measurement) &
            (original_df["Vehicle type"].isin([630,640,650]))
        ]
        .groupby("TimeGroup")
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        original_df[original_df["New_Measurement"] == measurement]
        .groupby("TimeGroup")
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(heavy_df, total_df, on="TimeGroup", how="left")
    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]

    return heavy_rate_df
###################################################################################################################


folder_path = r"C:\VISSIM_Workspace\network_new\network_version2\network_version2.3\network_version2.3.0\network03_mer"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]
mer_file = mer_list[0]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()


with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
        lines = file.readlines()
        # 데이터가 시작하는 인덱스 찾기
        data_start_idx = None

        for j, line in enumerate(lines):
            if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                data_start_idx = j
                break

        # 데이터프레임 생성
        if data_start_idx is not None:

            # 컬럼명 추출 및 공백 제거
            columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

            # 데이터 부분 추출 및 가공
            data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
            data = [line.strip().split(";") for line in data_lines if line.strip()]

            # 데이터프레임 생성
            df = pd.DataFrame(data, columns=columns)

            # 컬럼 내부 데이터 정수형 변환
            df = df.apply(pd.to_numeric, errors="coerce")

            original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


            #불필요 컬럼 제거
            original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

            original_df["New_Measurement"] = original_df["Measurem."] % 1000

            bins = np.arange(start_interval, end_interval+1, 300)
            labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

            # 구간 나누기 및 컬럼 추가
            original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

            # 평균속도
            speed_df = speed_mean(original_df)

            # 밀도
            density_df = density_mean(speed_df)

            # 가중 중차량 비율
            heavy_df = heavy_rate(original_df)


,TimeGroup,heavy_count,total_count,rate
0,1800~2100,7,150,0.046667
1,2100~2400,11,132,0.083333
2,2400~2700,3,147,0.020408
3,2700~3000,7,158,0.044304
4,3000~3300,11,163,0.067485
5,3300~3600,7,148,0.047297
6,3600~3900,12,150,0.080000
7,3900~4200,7,158,0.044304
8,4200~4500,4,142,0.028169
9,4500~4800,13,166,0.078313
